In [150]:
import pandas as pd
import numpy as np
import re
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm

In [151]:
matches = pd.read_csv('data_prepared.csv')
matches['match_id'] = range(matches.shape[0])

In [152]:
matches

,assists_p0,assists_p1,assists_p2,assists_p3,assists_p4,assists_p5,assists_p6,assists_p7,assists_p8,assists_p9,...,event_id,clutches_t1,clutches_t2,firstkills_t1,firstkills_t2,Map,time,t1_id,t2_id,match_id
0,3,6,1,0,1,3,5,2,4,1,...,2813.0,0.0,0.0,9.0,15.0,inferno,2017-05-11 13:10,4688,7397,0
1,3,3,4,4,4,2,4,6,2,2,...,2813.0,4.0,0.0,12.0,10.0,overpass,2017-05-11 13:10,7397,4688,1
2,3,3,7,3,5,6,6,2,5,4,...,2833.0,0.0,2.0,14.0,13.0,mirage,2017-05-11 13:00,6745,7897,2
3,5,3,4,4,5,3,3,4,0,3,...,2833.0,2.0,3.0,19.0,9.0,cobblestone,2017-05-11 13:00,7897,6745,3
4,5,4,2,3,4,5,0,1,5,4,...,2639.0,3.0,0.0,8.0,19.0,inferno,2017-05-11 05:20,5752,5973,4
5,1,4,5,2,2,0,3,2,4,1,...,2639.0,2.0,3.0,6.0,12.0,inferno,2017-05-11 05:20,6985,6673,5
6,4,4,3,3,7,5,1,3,0,4,...,2639.0,2.0,3.0,6.0,16.0,cobblestone,2017-05-11 04:30,6985,6673,6
7,4,1,4,4,2,2,0,4,1,3,...,2639.0,1.0,1.0,14.0,7.0,train,2017-05-11 04:25,5973,5752,7
8,1,3,3,5,2,4,3,8,4,7,...,2841.0,2.0,1.0,13.0,11.0,mirage,2017-05-11 04:00,7723,7289,8
9,0,2,5,3,1,4,2,1,0,4,...,2841.0,0.0,0.0,11.0,9.0,train,2017-05-11 04:00,7289,7723,9


In [153]:
offset = 5
for i in range(0, len(matches.columns), ofset):
    print(list(matches.columns[i: i+ofset]))

['assists_p0', 'assists_p1', 'assists_p2', 'assists_p3', 'assists_p4']
['assists_p5', 'assists_p6', 'assists_p7', 'assists_p8', 'assists_p9']
['adr_p0', 'adr_p1', 'adr_p2', 'adr_p3', 'adr_p4']
['adr_p5', 'adr_p6', 'adr_p7', 'adr_p8', 'adr_p9']
['deaths_p0', 'deaths_p1', 'deaths_p2', 'deaths_p3', 'deaths_p4']
['deaths_p5', 'deaths_p6', 'deaths_p7', 'deaths_p8', 'deaths_p9']
['fk_diff_p0', 'fk_diff_p1', 'fk_diff_p2', 'fk_diff_p3', 'fk_diff_p4']
['fk_diff_p5', 'fk_diff_p6', 'fk_diff_p7', 'fk_diff_p8', 'fk_diff_p9']
['kills_p0', 'headshots_p0', 'kills_p1', 'headshots_p1', 'kills_p2']
['headshots_p2', 'kills_p3', 'headshots_p3', 'kills_p4', 'headshots_p4']
['kills_p5', 'headshots_p5', 'kills_p6', 'headshots_p6', 'kills_p7']
['headshots_p7', 'kills_p8', 'headshots_p8', 'kills_p9', 'headshots_p9']
['rating_p0', 'rating_p1', 'rating_p2', 'rating_p3', 'rating_p4']
['rating_p5', 'rating_p6', 'rating_p7', 'rating_p8', 'rating_p9']
['p0_id', 'p1_id', 'p2_id', 'p3_id', 'p4_id']
['p5_id', 'p6_id', '

In [161]:
def get_team_players_ids(match, team=1, df=matches):
    """team = [1|2]"""
    players_ids = match[['p{}_id'.format(i) for i in range(10)]]
    players_teams_ids = match[['p{}_team_id'.format(i) for i in range(10)]]
    team_one_players_indexes = players_teams_ids==match['t{}_id'.format(team)]
    return list(players_ids[team_one_players_indexes.values])

def is_n_players_in_same_teams(players, team, n=5):
    return sum([p in team for p in players]) >= n

In [160]:
matches['t1_players_ids'] = [get_team_players_ids(match, team=1) for match_id, match in tqdm(matches.iterrows())]
matches['t2_players_ids'] = [get_team_players_ids(match, team=2) for match_id, match in tqdm(matches.iterrows())]

28172it [00:52, 538.69it/s]
28172it [00:50, 557.68it/s]


In [174]:
def get_all_matches_of_team(team, df=matches):
    players_in_t1 = df['t1_players_ids'].apply(lambda players: is_n_players_in_same_teams(players, team))
    players_in_t2 = df['t2_players_ids'].apply(lambda players: is_n_players_in_same_teams(players, team))
    
    return matches[players_in_t1 | players_in_t2]

In [175]:
match = matches.iloc[0]
t1 = get_team_players_ids(match, team=1)
t2 = get_team_players_ids(match, team=2)
t_wrong = get_team_players_ids(matches.iloc[-1], team=1)

(is_n_players_in_same_team(t1, match), 
 is_n_players_in_same_team(t2, match), 
 is_n_players_in_same_team(t_wrong, match))

1742it [00:20, 87.02it/s]

(True, True, False)

In [178]:
match = matches.iloc[0]
team = get_team_players_ids(match, team=2)
get_all_matches_of_team(team)

,assists_p0,assists_p1,assists_p2,assists_p3,assists_p4,assists_p5,assists_p6,assists_p7,assists_p8,assists_p9,...,clutches_t2,firstkills_t1,firstkills_t2,Map,time,t1_id,t2_id,match_id,t1_players_ids,t2_players_ids
0,3,6,1,0,1,3,5,2,4,1,...,0.0,9.0,15.0,inferno,2017-05-11 13:10,4688,7397,0,"[8804, 7964, 9278, 44, 9257]","[10761, 8917, 11219, 10762, 11217]"
1,3,3,4,4,4,2,4,6,2,2,...,0.0,12.0,10.0,overpass,2017-05-11 13:10,7397,4688,1,"[11219, 8917, 10761, 11217, 10762]","[8804, 7964, 9278, 44, 9257]"
30,5,4,1,4,2,3,3,4,1,3,...,0.0,12.0,10.0,train,2017-05-10 21:10,6896,7397,30,"[5385, 8324, 8242, 8549, 2826]","[8917, 10762, 11217, 10761, 11219]"
55,7,4,4,7,1,4,4,3,4,6,...,4.0,17.0,12.0,train,2017-05-10 18:20,7668,7397,55,"[566, 10054, 3373, 7868, 1038]","[11219, 10761, 11217, 8917, 10762]"
56,2,1,1,8,4,5,4,3,8,1,...,3.0,11.0,14.0,overpass,2017-05-10 18:20,7397,7668,56,"[11219, 10761, 11217, 8917, 10762]","[1038, 3373, 7868, 566, 10054]"
57,3,0,10,2,4,3,4,8,4,3,...,0.0,12.0,11.0,inferno,2017-05-10 18:20,7397,7668,57,"[10761, 11219, 8917, 11217, 10762]","[3373, 7868, 1038, 10054, 566]"
65,2,3,4,5,6,2,1,1,1,2,...,0.0,12.0,6.0,cobblestone,2017-05-10 16:20,7397,7615,65,"[11219, 10762, 8917, 11217, 10761]","[13319, 8935, 8530, 13317, 13318]"
66,4,5,6,3,4,2,2,6,2,6,...,0.0,11.0,13.0,train,2017-05-10 16:20,7397,7615,66,"[8917, 11219, 10762, 11217, 10761]","[13318, 8530, 13317, 13319, 8935]"
97,1,5,6,3,2,1,7,3,5,4,...,1.0,9.0,17.0,overpass,2017-05-09 21:45,4468,7397,97,"[322, 356, 4360, 7388, 329]","[10762, 11219, 10761, 11217, 8917]"
115,6,0,10,4,3,3,5,6,5,7,...,2.0,17.0,9.0,inferno,2017-05-09 15:15,7397,4468,115,"[10761, 11219, 8917, 10762, 11217]","[4360, 329, 322, 356, 7388]"
